# Inventarios SQLAlchemy

In [ ]:
# Cargar paquetes e iniciar sesion a la BD
from datetime import datetime
from tabulate import tabulate
from sqlalchemy.sql import func
from lib.database import SessionLocal
db = SessionLocal()

In [ ]:
# Cargar modelos
from plataforma_web.v1.abogados.models import Abogado
from plataforma_web.v1.autoridades.models import Autoridad
from plataforma_web.v1.centros_trabajos.models import CentroTrabajo
from plataforma_web.v1.distritos.models import Distrito
from plataforma_web.v1.domicilios.models import Domicilio
from plataforma_web.v1.edictos.models import Edicto
from plataforma_web.v1.funcionarios.models import Funcionario
from plataforma_web.v1.inv_categorias.models import InvCategoria
from plataforma_web.v1.inv_componentes.models import InvComponente
from plataforma_web.v1.inv_custodias.models import InvCustodia
from plataforma_web.v1.inv_equipos.models import InvEquipo
from plataforma_web.v1.inv_marcas.models import InvMarca
from plataforma_web.v1.inv_modelos.models import InvModelo
from plataforma_web.v1.inv_redes.models import InvRed
from plataforma_web.v1.listas_de_acuerdos.models import ListaDeAcuerdo
from plataforma_web.v1.listas_de_acuerdos_acuerdos.models import ListaDeAcuerdoAcuerdo
from plataforma_web.v1.materias.models import Materia
from plataforma_web.v1.materias_tipos_juicios.models import MateriaTipoJuicio
from plataforma_web.v1.modulos.models import Modulo
from plataforma_web.v1.oficinas.models import Oficina
from plataforma_web.v1.permisos.models import Permiso
from plataforma_web.v1.redams.models import Redam
from plataforma_web.v1.roles.models import Rol
from plataforma_web.v1.sentencias.models import Sentencia
from plataforma_web.v1.soportes_categorias.models import SoporteCategoria
from plataforma_web.v1.soportes_tickets.models import SoporteTicket
from plataforma_web.v1.usuarios.models import Usuario
from plataforma_web.v1.usuarios_roles.models import UsuarioRol

In [ ]:
# Consultar las oficinas
oficinas = db.query(Oficina.clave, Oficina.descripcion_corta).\
    filter(Oficina.estatus == "A").\
    order_by(Oficina.clave)
print(tabulate(oficinas.limit(4).all()))

In [ ]:
# Matriz con oficinas, usuarios, custodias, equipos, tipos, modelos y marcas
inv_equipos = db.\
    query(
        Oficina.clave,
        Usuario.email,
        InvCustodia.id.label("inv_custodia_id"), 
        InvCustodia.nombre_completo.label("inv_custodia_nombre_completo"), 
        InvEquipo.id.label("inv_equipo_id"),
        InvEquipo.tipo, 
        InvMarca.nombre.label("inv_marca_nombre"), 
        InvModelo.descripcion.label("inv_modelo_descripcion"), 
        InvEquipo.descripcion.label("inv_equipo_descripcion"),
        InvEquipo.fecha_fabricacion,
    ).\
    select_from(Oficina).join(Usuario, InvCustodia, InvEquipo, InvModelo, InvMarca).\
    filter(Oficina.estatus == "A").\
    filter(Usuario.estatus == "A").\
    filter(InvCustodia.estatus == "A").\
    filter(InvEquipo.estatus == "A").\
    order_by(InvCustodia.id, InvEquipo.id)
headers = ["Oficina", "Usuario", "ID Custodia", "Nombre", "ID Equipo", "Tipo", "Marca", "Modelo", "Descripcion", "Fabricacion"]
print(tabulate(inv_equipos.limit(8).all(), headers=headers))

In [ ]:
# Contar cantidad de equipos por tipo
consulta = db.\
    query(
        Oficina.clave,
        InvEquipo.tipo,
        func.count("*").label("cantidad"),
    ).\
    select_from(Oficina).join(Usuario, InvCustodia, InvEquipo).\
    filter(Oficina.estatus == "A").\
    filter(Usuario.estatus == "A").\
    filter(InvCustodia.estatus == "A").\
    filter(InvEquipo.estatus == "A").\
    order_by(InvEquipo.tipo).\
    group_by(Oficina.clave, InvEquipo.tipo)
print(tabulate(consulta.limit(8).all()))